<a href="https://colab.research.google.com/github/Conan486/Raivath_coding_challenge/blob/main/Assignment_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this assignment we are going to work with data that was downloaded using using the [yfinance](https://pypi.org/project/yfinance/) library. It containst daily closing prices of all symbols.

In [1]:
from multiprocessing   import Pool
from tqdm import *
import pandas as pd

In [2]:
#Question 1: Explain what the code below is trying to do. Describe each step.
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())


In [3]:
#Describe your answer below.
#Answer 1: Line by Line Explanation:

#1) NASDAQ Traded data in a CSV format is read from the link provided and stored as the pandas dataframe, "data".
#The delimiter (column seperator) of the CSV is "|".

#2) A new data dataframe called "data_clean" contains only the data where Test Issue is N.
#3) A set called symbols is created which contains all the symbols listed in the clean dataset from step 2. Symbols are unique due to using a set.

#4,5,6) Steps 1,2,3 are repeated using a different dataset called "otherlisted.txt" (possibly contains stocks listed in other exchanges). The same
#variables as above are reused, leading to overwriting the previously held values in them.

In [4]:
!wget  -O stocks.parquet https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1 # lets download the data

--2026-01-12 02:59:09--  https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/pm4vi37h059bx0du3ecj8/stocks.parquet?rlkey=8k31klc1tlf8bk4ocxpzbgm9c&dl=1 [following]
--2026-01-12 02:59:10--  https://www.dropbox.com/scl/fi/pm4vi37h059bx0du3ecj8/stocks.parquet?rlkey=8k31klc1tlf8bk4ocxpzbgm9c&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc08c347362a8324738774efe1eb.dl.dropboxusercontent.com/cd/0/inline/C4wrziOyc9dOdKnJ-BMn_nECWuTnPkYlogLEus9P14yMWmz9PSjcI4dOvu3tKkvGYxUbV8Wm3-tXmp0W6GKPbZTNoC6sTY5QzC6ldFqbo1b9A2_6MCgZnGJuEL10bhh6x8gvvUH_1c7gTI-H4PDGHKVz/file?dl=1# [following]
--2026-01-12 02:59:10--  https://uc08c347362a8324738774efe1eb.dl.dropboxuse

In [5]:
!ls stocks.parquet -l #downloaded file

-rw-r--r-- 1 root root 678640589 Jan 12 02:59 stocks.parquet


In [6]:
#Bonus question 1 - find on internet and write the command that will download the all data from the symbols list.
#data = yf.download(list(symbols), start="2025-01-01", end="2025-12-31")
#This downloads all data related to the symbols in the set/list "symbols" from last year.

In [7]:
data=pd.read_parquet('stocks.parquet')

In [8]:
data.head(1) # Below shows a sample of the data.
#date - describes the day of the price
#open - the price at which stock opened
#high - what was the intraday high
#low - what is the intraday low
#close - what was the closing price
#adj_close - the price calculated after adjusting for all future stock splits
#dividends - what are the dividends that were paid on the day.
#stock_splits - if the stock was split
#symbol the symbol of the stock


,date,open,high,low,close,adj_close,volume,dividends,stock_splits,symbol,day,year,month,dayofweek
0,1962-01-02,0.131273,0.131273,0.124177,0.124177,0.045278,2480333.0,0.0,0.0,HPQ,2,1962,1,Tuesday


In [9]:
# Question 2 find top 20 best performer stocks - i.e. the stocks that provided the best return.
# if the stocks adj_close on the earliest date was X and the adj_close on the 2021-01-01 is y and the number of days elapsed between these dates is Z.
# Then return = Y/X/Z
# Sort the stocks by this return and find the top 20
symbol_data = data.sort_values("date").groupby("symbol")

In [10]:
start = symbol_data.first()
end = symbol_data.last() # Using the last available date instead of "2021-01-01" to maximise Z.

In [11]:
result = pd.merge(start, end, on = "symbol", suffixes=('_start', '_end'))
result = result[["adj_close_start", "adj_close_end", "date_start", "date_end"]]

# Converting strings to datetime objects
result["date_start"] = pd.to_datetime(result["date_start"])
result["date_end"] = pd.to_datetime(result["date_end"])
result["Total days"] = (result["date_end"] - result["date_start"]).dt.days

# Filtering one-day stocks and initial zero dollar value stocks
result = result[result["Total days"] > 0]
result = result[result["adj_close_start"] > 0]

result["return"] = result["adj_close_end"] / result["adj_close_start"] / result["Total days"]
result.sort_values("return", ascending=False).head(20)

,adj_close_start,adj_close_end,date_start,date_end,Total days,return
symbol,,,,,,
NVO,2.275794e-09,111.629997,1981-04-30,2021-12-27,14851,3.302876e+06
NKE,3.775067e-06,167.580002,1980-12-02,2021-12-27,15000,2.959418e+03
DCI,9.812100e-06,57.939999,1980-03-17,2021-12-27,15260,3.869564e+02
TJX,2.857077e-05,74.290001,1987-06-26,2021-12-27,12603,2.063167e+02
WTRG,4.768529e-04,52.549999,1980-03-17,2021-12-27,15260,7.221605e+00
TTC,1.051490e-03,99.879997,1980-03-17,2021-12-27,15260,6.224708e+00
USB,7.828808e-04,56.740002,1973-05-03,2021-12-27,17770,4.078555e+00
MCD,3.923203e-03,268.239990,1966-07-05,2021-12-27,20264,3.374097e+00
PII,3.345692e-03,107.000000,1987-09-16,2021-12-27,12521,2.554223e+00


In [12]:
#Question 3: Create a copy of this notebook called assignment-aws and follow the instructions in readme to run the notebook on AWS compute instance.
# Compare performance of running natively on colab and colab+AWS
# 2 mins 12 seconds in total to run in colab
# The AWS+Colab version was slower to run taking 10 mins 46 secs to run. The biggest bottlenecks were reading the parquet data and the groupby function
# in both versions, but it took significantly longer to do the same in the AWS+Colab version.